# Georcoronal Correction


In [4]:
# import
%matplotlib inline
import os,sys
import matplotlib
import numpy as np
from astropy.io import fits
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
import csv
import pandas as pd
import sfdmap
import extinction
from astropy import units as u
import math
import pylatex

# pyqsofit
from scipy import interpolate

# for SDSS SQL queries
import mechanize
from io import BytesIO

import glob,timeit

#sys.path.append('/Users/runnojc1/Software/py/pyspeckit')
import pyspeckit
from pyspeckit.spectrum.models.inherited_gaussfitter import gaussian
from pyspeckit.specwarnings import PyspeckitWarning
from astropy.table import Table

In [8]:
## Reading table to get the table header
file_path = 'HET/combined_spectrum_20230113_0000022_exp01.fits'

with fits.open(file_path) as hdulist:
    hdulist.info()
    hdu_header = hdulist[0].header
    print("HDU Header:")
    print(repr(hdu_header))
    


Filename: HET/combined_spectrum_20230113_0000022_exp01.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU       4   ()      
  1                1 BinTableHDU     18   8256R x 5C   [D, D, D, D, D]   
HDU Header:
SIMPLE  =                    T / conforms to FITS standard                      
BITPIX  =                    8 / array data type                                
NAXIS   =                    0 / number of array dimensions                     
EXTEND  =                    T                                                  


## Find the width of the lines in pixels and Angstroms.


## Projected Aperture Calculation

### Given Data
- **Aperture PSA width:** 0.2 arcseconds
- **COS Platescale:** 23.0 milliarcseconds/pixel = 0.023 arcseconds/pixel
- **Grating G140L Average Dispersion:** 0.6 Angstroms/pixel
- **Central Wavelength:** 800 Angstrom
- 

### Calculation
1. **Calculate the projected aperture width in pixels:**

   $$
   \text{Projected aperture width (pixels)} = \frac{0.2 \text{ arcseconds}}{0.023 \text{ arcseconds per pixel}} = 8.7 \text{ pixels}
   $$

2. **Calculate the projected aperture width in Angstroms:**

   $$
   \text{Projected aperture width (Angstroms)} = 8.7 \text{ pixels} \times 0.6 \text{ Angstroms per pixel} = 5.22 \text{ Angstroms}
   $$

